<font color='tomato'><font color="#CC3D3D"><p>
# A Simple DNN Model Ensemble
- Make different DNN models by changing only the random seed values
- Ensemble the models using the power mean

##### Import modules

In [1]:
import pandas as pd
import numpy as np
import os
import random
import pickle
from tqdm import tqdm
from IPython.display import Image, clear_output
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

2.4.1


In [2]:
with open('../input/train_selected_0614_2.pkl', 'rb') as f:  # cv.pkl이라는 파일을 바이너리 읽기(rb)모드로 열어서 f라 하고
    data = pickle.load(f)

In [3]:
with open('../input/test_selected_0614_2.pkl', 'rb') as f:  # cv.pkl이라는 파일을 바이너리 읽기(rb)모드로 열어서 f라 하고
    data_te = pickle.load(f)

In [4]:
y_train = pd.read_csv('../input/y_train.csv').age

In [5]:
data.shape, data_te.shape, y_train.shape

((21587, 424), (14380, 424), (21587,))

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5931256831258100406
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1408047310
locality {
  bus_id: 1
  links {
  }
}
incarnation: 658988843175493956
physical_device_desc: "device: 0, name: GeForce MX250, pci bus id: 0000:3a:00.0, compute capability: 6.1"
]


##### Set random seeds to make your results reproducible

In [9]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(s1,s2,s3, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(s1)
    random.seed(s2)
    tf.compat.v1.set_random_seed(s3)
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # for GPU
#    print("RANDOM SEEDS RESET")  # optional

### Step 1: Load and process the data

##### Read data

In [10]:
# 1st round 2등팀의 훈련/평가/적용 데이터를 읽어온다.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y_train, test_size=0.3, random_state=0)
# 모델링에 사용되는 최종 학습 및 평가 데이터 확인
X_train.shape, X_test.shape

((15110, 424), (6477, 424))

##### Feature scaling

In [10]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

##### Split data into train & validation set 

In [11]:
# Train/Validation
i = int(round(X_train.shape[0] * 0.8,0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

### Step 2: Define the hyper-model

In [12]:
def model_fn(hp):
    inputs = keras.Input(shape=(X_train.shape[1],))
    x = inputs
    for i in range(hp.Int('num_layers', 2, 3)):
        x = keras.layers.Dense(hp.Int('unit_'+ str(i), 16, 64, step=16), activation='swish')(x)
        x = keras.layers.Dropout(hp.Float('dropout_'+str(i), 0, 0.5, step=0.25, default=0.5))(x)
    outputs = keras.layers.Dense(1, activation='linear')(x)
    model = keras.Model(inputs, outputs)
    model.compile(loss='mse', 
                    optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                    metrics=[keras.metrics.RootMeanSquaredError()])
    return model

### Step 3: Build multiple hyper-tuned models

In [13]:
N = 5
rmses = []
preds = []

with tf.device('/device:GPU:0'):
    for i in tqdm(range(N)):
        reset_seeds(i,i*10,i*100)
        tuner = kt.Hyperband(model_fn,
                         objective=kt.Objective('val_root_mean_squared_error', direction="min"), 
                         max_epochs=50,
                         hyperband_iterations=5,
                         overwrite=True,
                         directory='dnn_tuning')
        tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
                 callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)], verbose=0)
        model = tuner.get_best_models(1)[0]
        rmses.append(model.evaluate(X_test, y_test)[1])
        preds.append(model.predict(data_te).flatten())         

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

INFO:tensorflow:Oracle triggered exit
203/203 [==============================] - 0s 1ms/step - loss: 71.3012 - root_mean_squared_error: 8.4434


 20%|████████████████                                                                | 1/5 [41:17<2:45:10, 2477.54s/it]

INFO:tensorflow:Oracle triggered exit
203/203 [==============================] - 0s 1ms/step - loss: 71.4206 - root_mean_squared_error: 8.4501


 40%|███████████████████████████████▏                                              | 2/5 [1:17:44<1:59:31, 2390.34s/it]

...

 60%|██████████████████████████████████████████████▊                               | 3/5 [1:53:57<1:17:30, 2325.31s/it]

INFO:tensorflow:Oracle triggered exit
203/203 [==============================] - 0s 1ms/step - loss: 69.0632 - root_mean_squared_error: 8.3099


 80%|████████████████████████████████████████████████████████████████                | 4/5 [2:32:35<38:43, 2323.02s/it]

INFO:tensorflow:Oracle triggered exit
203/203 [==============================] - 0s 1ms/step - loss: 69.8941 - root_mean_squared_error: 8.3594


100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [3:14:38<00:00, 2335.70s/it]


### Step 4: Ensemble models & make submissions

In [14]:
print('\nEvaluation Summary:')
rmses = pd.Series(rmses)
print(rmses.sort_values())
print('mean={:.5f}, std={:.3f}'.format(rmses.mean(), rmses.std()))   


Evaluation Summary:
3    8.360998
4    8.419109
0    8.503122
2    8.505464
1    8.524975
dtype: float64
mean=8.46273, std=0.070


In [15]:
# Power mean ensemble
THRESHOLD = 8.75  # 이 값보다 성능이 나쁜 prediction은 버림.
p = 1
pred = 0
n = 0
for i in range(N):
    if rmses.iloc[i] < THRESHOLD:
        pred = pred + preds[i]**p 
        n += 1
pred = pred / n    
pred = pred**(1/p)

In [16]:
# Make submissions: 앙상블 (전)8.59847 => (후)8.54436
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
sub = pd.read_csv(os.path.abspath("../input")+'/submission_test.csv')
sub['age'] = pred
sub.to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_06150309.csv' is ready to submit.


<font color="#CC3D3D"><p>
# End